In [1]:
import $file.common
import spark._
import common._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{IntegerType, StringType, StructType}
import spark.implicits._
import spark.sqlContext.implicits._
import org.apache.spark.sql.functions.{col, to_date}
import org.apache.spark.sql.types.DateType

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/07/09 15:53:00 INFO SparkContext: Running Spark version 2.4.5
21/07/09 15:53:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/07/09 15:53:01 INFO SparkContext: Submitted application: Queries Optimization
21/07/09 15:53:01 INFO SecurityManager: Changing view acls to: jovyan
21/07/09 15:53:01 INFO SecurityManager: Changing modify acls to: jovyan
21/07/09 15:53:01 INFO SecurityManager: Changing view acls groups to: 
21/07/09 15:53:01 INFO SecurityManager: Changing modify acls groups to: 
21/07/09 15:53:01 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
21/07/09 15:53:02 INFO Utils: Successfully started service 'sparkDriver' on port 3

21/07/09 15:53:03 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0-sources.jar at spark://98858bd2f6bb:38125/jars/scala-collection-compat_2.12-2.0.0-sources.jar with timestamp 1625845983352
21/07/09 15:53:03 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0.jar at spark://98858bd2f6bb:38125/jars/scala-collection-compat_2.12-2.0.0.jar with timestamp 1625845983352
21/07/09 15:53:03 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.2.0/scala-xml_2.12-1.2.0-sources.jar at spark://98858bd2f6bb:38125/jars/scala-xml_2.12-1.2.0-sources.jar with timestamp 1625845983352
21/07/09 15:53:03 INFO SparkContext: Added JAR file:/home/jovyan/.cache

21/07/09 15:53:03 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-scala_2.12/2.9.9/jackson-module-scala_2.12-2.9.9.jar at spark://98858bd2f6bb:38125/jars/jackson-module-scala_2.12-2.9.9.jar with timestamp 1625845983365
21/07/09 15:53:03 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-paranamer/2.9.9/jackson-module-paranamer-2.9.9.jar at spark://98858bd2f6bb:38125/jars/jackson-module-paranamer-2.9.9.jar with timestamp 1625845983365
21/07/09 15:53:03 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/com/squareup/okhttp3/logging-interceptor/3.11.0/logging-interceptor-3.11.0.jar at spark://98858bd2f6bb:38125/jars/logging-interceptor-3.11.0.jar with timestamp 1625845983366
21/07/09 15:53:03 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.

Spark UI

import $file.$     

import spark._

import common._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types.{IntegerType, StringType, StructType}

import spark.implicits._

import spark.sqlContext.implicits._

import org.apache.spark.sql.functions.{col, to_date}

import org.apache.spark.sql.types.DateType

In [2]:
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

In [3]:
spark.conf.set("spark.sql.shuffle.partitions", 8)

# Query 5: Tasa de infecciones frente a vacunaciones

Para obtener esta consulta se van a utilizar 3 datasets diferentes, en primer lugar un dataset de infecciones de Covid19 actualizado a las fechas de vacunaciones. El dataset de datos demográficos con el que ya veníamos trabajando y por último un dataset con el número de vacunaciones diarias por país.

El objetivo de esta query es limpiar los datos y ajustar los tipos fecha para:
    - Obtener la tasa de infecciones por densidad de población (como hemos hecho anteriormente).
    - Obtener la tasa de vacunaciones por densidad de población.
    - Obtener la tasa de vacunaciones frente a infecciones.
    
De esta forma podremos observar que crece más rápido, si las vacunas o las infecciones, y que porcentaje de la población está pendiente de ser vacunada.

## 1. Consulta con DataSet

modifico los datos de entrada para que se ajuste la fecha

In [4]:
def infectionsDS = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/covid_19_data.csv")
    .withColumn("ObservationDate",translate($"ObservationDate","/","-"))
    .withColumn("ObservationDate",to_date(col("ObservationDate"),"MM-dd-yyyy"))
    .withColumnRenamed("ObservationDate","date")
    .as[(Int,String,String,String,String,Double,Double,Double)]

defined function infectionsDS

In [5]:
def populationDS = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/population_by_country_2020.csv")
    .withColumnRenamed("Country (or dependency)","Country")
    .withColumnRenamed("Population (2020)","Population")
    .as[(String,Float,String,Float,Float,Float,Double,String,String,String,String)]

defined function populationDS

In [6]:
def vaccinationsDS = spark.read
    .option("header", "true")
    .option("charset", "UTF8")
    .option("delimiter",",")
    .option("inferSchema", "true")
    .csv("../datasets/country_vaccinations.csv")
    .withColumn("date",to_date(col("date"),"MM-dd-yyyy"))
    .withColumnRenamed("date","dateVaccinated")
    .as[(String,String,java.sql.Timestamp,Double,Double,Double,Double,Double,Double,Double,Double,Double,String,String,String)]

defined function vaccinationsDS

In [7]:
def megaDS = infectionsDS
    .join(
        vaccinationsDS,$"date" === $"dateVaccinated"
        && $"Country/Region" === $"country"
    ).join(populationDS,"Country")
        .na.fill(0)
        .select($"country",
                $"date",
                $"confirmed",
                $"people_vaccinated",
                $"Population",
                $"confirmed" / $"Population" as "infection Per Population",
                $"people_vaccinated"/ $"Population" as "vaccination Per Population",
                $"people_vaccinated" / $"confirmed" as "infection-vaccination rate")
        .orderBy($"date".asc)
        .withColumn("infection-vaccination rate", round($"infection-vaccination rate",8))
        .withColumn("vaccination Per Population", round($"vaccination Per Population",8))
        .as[(String,java.sql.Timestamp,Double,Double,Int,Double,Double,Option[Double])]

defined function megaDS

## 2. Consulta con DataFrame

In [8]:
def infectionsDF = spark.read
    .option("header", "true")
    .csv("../datasets/covid_19_data.csv")
    .withColumn("ObservationDate",translate($"ObservationDate","/","-"))
    .withColumn("ObservationDate",to_date(col("ObservationDate"),"MM-dd-yyyy"))
    .withColumnRenamed("ObservationDate","date")
    .withColumnRenamed("Country/Region","CountryInfection")

defined function infectionsDF

In [9]:
def populationDF = spark.read
    .option("header", "true")
    .csv("../datasets/population_by_country_2020.csv")
    .withColumnRenamed("Country (or dependency)","Country")
    .withColumnRenamed("Population (2020)","Population")

defined function populationDF

In [10]:
def vaccinationsDF = spark.read
    .option("header", "true")
    .csv("../datasets/country_vaccinations.csv")
    .withColumnRenamed("date","dateVaccinated")
    .withColumnRenamed("country","CountryVaccination")

defined function vaccinationsDF

In [11]:
def megaDF = infectionsDF
    .join(
        vaccinationsDF,$"date" === $"dateVaccinated"
        && $"CountryInfection" === $"CountryVaccination"
    ).join(populationDF,$"Country" === $"CountryInfection")
        .na.fill(0)
        .select($"CountryInfection",
                $"date",
                $"confirmed",
                $"people_vaccinated",
                $"Population",
                $"confirmed" / $"Population" as "infection Per Population",
                $"people_vaccinated"/ $"Population" as "vaccination Per Population",
                $"people_vaccinated" / $"confirmed" as "infection-vaccination rate")
        .orderBy($"date".asc)
        .withColumn("infection-vaccination rate", round($"infection-vaccination rate",8))
        .withColumn("vaccination Per Population", round($"vaccination Per Population",8))

defined function megaDF

## 3. Visualización de rendimiento

In [13]:
val (x, y) = Seq(
    "DataSet" -> runWithOutput(megaDS.collect),
    "DataFrame" -> runWithOutput(megaDF.collect),
).unzip

Bar(x, y).plot()

csv at cmd3.sc:6

1 / 1

csv at cmd3.sc:6

5 / 5

csv at cmd5.sc:6

...

5 / 5

collect at cmd12.sc:3

8 / 8

Took 2734


x: Seq[String] = List("DataSet", "DataFrame")
y: Seq[Int] = List(4442, 2734)
res12_1: String = "plot-e1e6d324-b589-4e24-bcf2-312541b6c125"

## 4. Comparativas de rendimiento

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    megaDS.collect
    )

In [ ]:
ch.cern.sparkmeasure.StageMetrics(spark).runAndMeasure(
    megaDF.collect
)

## 5. Visualización de datos con plotly

### crecimiento de la vacunacion con respecto a la densidad población

In [ ]:
val y = megaDF
    .filter($"country" === "Chile")
    .select($"vaccination Per Population" * 10000000)
    .collect
    .map(r => r(0).toString.toDouble)
    .toList

val x = megaDF
    .filter($"country" === "Chile")
    .select($"date")
    .collect
    .toList
    .map(_.toString)

val y1 = megaDF
    .filter($"country" === "Chile")
    .select($"people_vaccinated")
    .collect
    .map(r => r(0).toString.toDouble)
    .toList

val x1 = megaDF
    .filter($"country" === "Chile")
    .select($"date")
    .collect
    .toList
    .map(_.toString)

val data = Seq(
        Scatter(x,y).withName("Population vaccinated"),
        Scatter(x1,y1).withName("Vaccines administrated"))
    .map(_.withFill(Fill.ToNextY)
    .withStackgroup("A"))

val myLayout =
    Layout()
    .withTitle("CHILE")

plot(data,myLayout)